Import required packages:

In [1]:
from sqlalchemy import create_engine
import pandas as pd
import yaml
from statsmodels.graphics.gofplots import qqplot
from scipy import stats
from scipy.stats import yeojohnson
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

Import data from RDS database:

In [10]:
# Create a function which loads the yaml credentials file and return 
# the data dictionary contained within
def import_yaml():
    with open('credentials.yaml', 'r') as file:
        return yaml.safe_load(file)
yaml_dictionary = import_yaml() 

# Create a class which contains methods we will use to 
# extract data from the RDS database
class RDSDatabaseConnector:
    def __init__(self, yaml_dictionary):
        self.yaml_dictionary = yaml_dictionary
        
    # Define a method which initialises a SQLAlchemy engine 
    # from the credentials provided to the class
    def connect_database(self):
        DATABASE_TYPE = 'postgresql'
        DBAPI = 'psycopg2'
        ENDPOINT = self.yaml_dictionary['RDS_HOST']
        USER = self.yaml_dictionary['RDS_USER']
        PASSWORD = self.yaml_dictionary['RDS_PASSWORD']
        PORT = self.yaml_dictionary['RDS_PORT']
        DATABASE = self.yaml_dictionary['RDS_DATABASE']
        return create_engine(f'{DATABASE_TYPE}+{DBAPI}://{USER}:{PASSWORD}@{ENDPOINT}:{PORT}/{DATABASE}')
            
    # Define a method which extracts data from the RDS 
    # database and returns it as a Pandas DataFrame
    def extract_data(self, engine):
        return pd.read_sql_table('customer_activity', engine)
    
# Use the methods to extract data
customer_activity_class = RDSDatabaseConnector(yaml_dictionary)
engine = customer_activity_class.connect_database()
customer_activity = customer_activity_class.extract_data(engine)



Convert data types:

In [19]:
# Get the data type for each column
print('Data types before the transformation:')
print(customer_activity.dtypes)
# Create a class which contains methods that can be appplied
# to the columns to handle data type conversions
class ColumnTransform:
    def __init__(self, dataframe): 
        self.dataframe = dataframe
        
    # Define a method which converts columns to category datatype
    def convert_to_category(self, column):
        self.dataframe[column] = self.dataframe[column].astype('category')
         

# Covert data types for some columns to category 
customer_activity_DataTransform = ColumnTransform(customer_activity)
columns_to_transform = ['administrative', 'product_related', 'informational', 
                        'month', 'operating_systems', 'browser', 'region', 
                        'traffic_type', 'visitor_type']
for column in columns_to_transform:
    customer_activity_DataTransform.convert_to_category(column)
    
# Check if the columns have been transformed correctly    
print('Data types after the transformation:')
print(customer_activity.dtypes)

Data types before the transformation:
administrative              category
administrative_duration      float64
informational               category
informational_duration       float64
product_related             category
product_related_duration     float64
bounce_rates                 float64
exit_rates                   float64
page_values                  float64
month                       category
operating_systems           category
browser                     category
region                      category
traffic_type                category
visitor_type                category
weekend                         bool
revenue                         bool
dtype: object
Data types after the transformation:
administrative              category
administrative_duration      float64
informational               category
informational_duration       float64
product_related             category
product_related_duration     float64
bounce_rates                 float64
exit_rates             

Following columns have been converted to category data type:
- administrative
- informational
- product_related
- month
- operating_systems
- browser
- region
- traffic_type
- visitor_type

Create a class which contains methods to extract information from the DataFrame and its columns:

In [21]:
# Create a class which contains methods to extract information
# from the DataFrame and its columns
class DataFrameInfo:
    def __init__(self, dataframe):
        self.dataframe = dataframe
        
    # Define a method to describe all columns in the DataFrame
    def get_description(self):
        print('Description for columns:')
        print(self.dataframe.describe())
        
    # Define a method to extract statistical values (mean, median
    # and standard deviation) from the float64 columns
    def get_statistical_values(self, column):
        print(f'Mean for {column}:', self.dataframe[column].mean())
        print(f'Median for {column}:', self.dataframe[column].median())
        print(f'Standard Deviation for {column}:', self.dataframe[column].std())
        
    # Define a method to count distinct values in categorical columns
    def get_counts(self, column):
        self.dataframe[column].value_counts()
        
    # Define a method to print out the shape of the DataFrame
    def get_shape(self):
        print('Shape of the DataFrame:', self.dataframe.shape)
        
    # Define a method to generate a count and percentage of NULL values 
    # in each column
    def get_NULL_counts(self, column):
        print(f'Number of NULLs for {column}:', self.dataframe[column].isnull().sum())
        print(f'Percentage of NULLs for {column}:', round(self.dataframe[column].isnull().sum()/
              len(self.dataframe) * 100, 2), '%')

Create a class to visualise insights from the DataFrame:

In [22]:
class Plotter:
    def __init__(self, dataframe):
        self.dataframe = dataframe
    
    # Define a method to plot histogram to check skewness
    def plot_histograms(self, column):
        self.dataframe[column].hist(bins = 3)
        plt.title(f'Histogram for {column}')
        plt.show()
        
    # Define a method to plot Q-Q plot to check skewness
    def plot_qqplot(self, column):
        qqplot(self.dataframe[column], scale = 1, line = 'q', fit = True)
        plt.title(f'Q-Q plot for {column}')
        plt.show()
        
    # Define a method to plot box plot to check outliers
    def plot_boxplot(self, column):
        sns.boxplot(y = self.dataframe[column], color = 'lightgreen', showfliers = True)
        plt.title(f'Box plot with scatter points of {column}')
        plt.show()
        
    # Define a method to plot correlation matrix
    def plot_correlation_matrix(self):
        sns.heatmap(self.dataframe.corr(), annot = True, cmap = 'coolwarm')
        plt.show()

Create a class to perform EDA transformations on the DataFrame:

In [ ]:
class DataFrameTransform:
    def __init__(self, dataframe):
        self.dataframe = dataframe
        
    # Define a method to impute NULLs with mode
    def mode_impute(self, column):
        column_mode = self.dataframe[column].mode().iloc[0]
        self.dataframe[column].fillna(column_mode, inplace=True)
        
    # Define a method to impute NULLs with mean
    def mean_impute(self, column):
        self.dataframe[column] = self.dataframe[column].fillna(self.dataframe[column].mean())
        
    # Define a method to impute NULLs with median
    def median_impute(self, column):
        self.dataframe[column] = self.dataframe[column].fillna(self.dataframe[column].median())
        
    # Define a method to compute log transform
    def log_transform(self, column):
        self.dataframe[column] = self.dataframe[column].map(lambda i: np.log(i) if i > 0 else 0)
        
    # Define a method to compute Yeo-Johnson transform
    def yeo_johnson_transform(self, column):
        self.dataframe[column] = stats.yeojohnson(self.dataframe[column])[0]
        
    # Define a method to drop outliers
    def drop_outliers(self, column):
        Q1 = self.dataframe[column].quantile(0.25)
        Q3 = self.dataframe[column].quantile(0.75)
        IQR = Q3 - Q1
        lower = Q1 - 1.5 * IQR
        upper = Q3 + 1.5 * IQR
        outliers = self.dataframe[(self.dataframe[column] >= Q1 - 1.5*IQR) & (self.dataframe[column] <= Q3 + 1.5*IQR)]
        self.dataframe = self.dataframe.drop(outliers.index)

Put columns into different groups based on data types:

In [ ]:
all_columns = customer_activity.columns
numeric_columns = ['administrative_duration', 'informational_duration', 'product_related_duration',
                   'bounce_rates', 'exit_rates', 'page_values']
categorical_columns = ['administrative', 'informational', 'product_related', 'month',
                       'operating_systems', 'browser', 'region', 'visitor_type']
boolean_columns = ['weekend', 'revenue']

Create instances of the classes: